# Shopify App Store Capstone Project

## Objectives

This notebook will be used to gather and organize data for my Shopify App Store Capstone Project

### 1. Sourcing and Loading
#### 1.1 Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.max_columns = None

#### 1.2 Loading The Raw Data
Data sources:
    https://www.kaggle.com/datasets/usernam3/shopify-app-store
    

In [2]:
# Location of invididual CSV files being used
csv_dict = {
    'apps':"/Users/msmith/Shopify_Analysis_Capstone/data/raw/apps.csv",
    'apps_categories':"/Users/msmith/Shopify_Analysis_Capstone/data/raw/apps_categories.csv",
    'categories':"/Users/msmith/Shopify_Analysis_Capstone/data/raw/categories.csv",
    'key_benefits':"/Users/msmith/Shopify_Analysis_Capstone/data/raw/key_benefits.csv",
    'pricing_plan_features':"/Users/msmith/Shopify_Analysis_Capstone/data/raw/pricing_plan_features.csv",
    'pricing_plans':"/Users/msmith/Shopify_Analysis_Capstone/data/raw/pricing_plans.csv",
    'reviews':"/Users/msmith/Shopify_Analysis_Capstone/data/raw/reviews.csv"
}

# Reading each file into a dataframe & storing in a dictionary
df_dict = {}

for k, v in csv_dict.items():
    df = pd.read_csv(v)
    df_dict[k] = df

df_dict_keys = list(df_dict.keys())
print(df_dict_keys)



['apps', 'apps_categories', 'categories', 'key_benefits', 'pricing_plan_features', 'pricing_plans', 'reviews']


### 2. Cleaning, Transforming, and Visualizing Data
#### 2.1 Exploring the Data



In [73]:

for index, (k, v) in enumerate(df_dict.items()):
    print("DataFrame " + str(index + 1) + " of 7: " + k)
    print(v.shape)
    print(v.columns)
    print(v.index)
    print(v.isnull().sum())
    print(v.dtypes)
    print("----------------------------")


DataFrame 1 of 7: apps
(4750, 12)
Index(['id', 'url', 'app_title', 'developer', 'developer_link', 'icon',
       'rating', 'reviews_count', 'description_raw', 'description', 'tagline',
       'pricing_hint'],
      dtype='object')
RangeIndex(start=0, stop=4750, step=1)
id                    0
url                   0
app_title             0
developer             0
developer_link        0
icon                  0
rating                0
reviews_count         0
description_raw       0
description           0
tagline               0
pricing_hint       1980
dtype: int64
id                  object
url                 object
app_title           object
developer           object
developer_link      object
icon                object
rating             float64
reviews_count        int64
description_raw     object
description         object
tagline             object
pricing_hint        object
dtype: object
----------------------------
DataFrame 2 of 7: apps_categories
(7376, 2)
Index(['id', 'cate

In [4]:
#Unpacking dictionary into separate dataframes
apps_df = df_dict['apps']
apps_categories_df = df_dict['apps_categories']
categories_df = df_dict['categories']
key_benefits_df = df_dict['key_benefits']
pricing_plan_features_df = df_dict['pricing_plan_features']
pricing_plans_df = df_dict['pricing_plans']
reviews_df = df_dict['reviews']

#### 2.2 Cleaning Data & Merging DataFrames
##### 2.2.1 Cleaning Tables

In [5]:
#apps_df
#renaming columns
apps_df.rename(columns={'title':'app_title'}, inplace=True)

In [6]:
#apps_categories_df & categories_df
#renaming columns
apps_categories_df.rename(columns={'app_id':'id'}, inplace=True)
categories_df.rename(columns={'id':'category_id','title':'app_category'}, inplace=True)

#merging tables to reduce future redundancy
categories= apps_categories_df.merge(categories_df, how='left', on = 'category_id')
categories.drop('category_id', axis=1, inplace=True)

print(categories.shape)
categories.head()

(7376, 2)


,id,app_category
0,9e4748a9-7eda-4814-83b6-0537d44152b1,Store design
1,d1476138-a608-4bb9-8d39-b30f3ca7617d,Store design
2,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,Store design
3,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,Customer support
4,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,Store design


In [7]:
categories.app_category.value_counts()

Store design                         2087
Sales and conversion optimization    1294
Marketing                            1068
Orders and shipping                   858
Customer support                      409
Reporting                             350
Inventory management                  342
Productivity                          282
Finding and adding products           252
Finances                              163
Trust and security                    144
Places to sell                        127
Name: app_category, dtype: int64

In [8]:
#Consolidating dataframe to have a 1 to 1 relationship between app_id & category
#Apps with multiple categories have been concatenated into one hybrid category
grouped_categories = categories.groupby("id", as_index=False).agg({'app_category': lambda category : ', '.join(category)})
print(grouped_categories.shape)
grouped_categories.head()

(4750, 2)


,id,app_category
0,00014403-ad17-4c9b-bbcf-5adc27ba18a9,"Inventory management, Reporting"
1,000270a8-af0b-4862-82df-25decbcc8d86,Store design
2,0006d5af-c461-4bdb-b32e-a4fecd00846b,Store design
3,001ce7ab-14dc-45ce-98f1-1b42b61c86c1,"Store design, Sales and conversion optimization"
4,00215fb3-ff7c-4111-8f0c-8e7e2a9027fb,Marketing


In [9]:
#key_benefits_df
#renaming columns and reviewing status
key_benefits_df.rename(columns={'app_id':'id', 'title':'benefit_title', 'description':'benefit_description'}, inplace=True)
print(key_benefits_df.shape)
key_benefits_df.head()

(12927, 3)


,id,benefit_title,benefit_description
0,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,FAQ Accordion Product Info,Add a custom FAQ accordion with answers to com...
1,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,FAQ Accordion Refund Policy,Give your customers answers to refund policy q...
2,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,FAQ Accordion Page Builder,Reduce support time and answer common question...
3,9e4748a9-7eda-4814-83b6-0537d44152b1,Translate your store easily,Add 100+ languages in your store. Increase you...
4,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,Improve Conversions & Revenue,"Wheel of Fortune, Currency Converter, Animated..."


In [10]:
key_benefits_df.id.value_counts()

d6e49a3c-2f9f-4bfa-8c26-5d024faf2241    3
81608e15-e157-44c8-a5f0-e0864343c55f    3
33d11be2-3b01-44ce-94c3-2a8b5ef40a60    3
b8ff16ee-67a2-4394-95ce-aca15825b08e    3
5b01e3f2-770d-4530-b9aa-acb509d141f2    3
                                       ..
0ba314ac-1e1b-4740-86d2-5acc6220fad7    3
41ec2fb0-d4d8-40ab-b62c-213a19996ccf    3
4b8ab10d-d2ef-4af2-9af5-a98d0ae4b6ae    3
881fe88d-78cd-45e0-b013-291a9d287ee5    3
17918ea4-80b9-4811-947d-c0b2f950976a    3
Name: id, Length: 4309, dtype: int64

In [11]:
#Consolidating dataframe to have a 1 to 1 relationship between app_id, benefit_title & benefit_description
#Apps with multiple benefit_title or benefit_description have been concatenated into one hybrid category
grouped_key_benefits = key_benefits_df.groupby("id", as_index=False).agg(
    {'benefit_title': lambda bt : ' || '.join(bt), 
     'benefit_description': lambda bd : ' || '.join(bd)
    })
print(grouped_key_benefits.shape)
grouped_key_benefits.head()

(4309, 3)


,id,benefit_title,benefit_description
0,00014403-ad17-4c9b-bbcf-5adc27ba18a9,Low Inventory alerts || Sales Forecasting || A...,Get an email report with the list of items tha...
1,000270a8-af0b-4862-82df-25decbcc8d86,AI Search || Conversion Rate Optimization || R...,"Advance Search Solution with AI search, Image..."
2,0006d5af-c461-4bdb-b32e-a4fecd00846b,Available for Kaleido Members || Link Preloadi...,You know the password? If you're a member and ...
3,001ce7ab-14dc-45ce-98f1-1b42b61c86c1,"Out of the box, Boost sales! || AI-driven Rule...","AI-driven exit intent pop up, boost sales！Out-..."
4,00215fb3-ff7c-4111-8f0c-8e7e2a9027fb,Get Better Sales & Engagement || Easy-To-Use F...,Increase your email marketing engagement and c...


In [12]:
#pricing_plan_features_df & pricing_plans_df
#renaming columns & reviewing status
pricing_plan_features_df.rename(columns={'pricing_plan_id':'id', 'app_id':'pricing_id','feature':'pricing_plan_feature'}, inplace=True)
pricing_plans_df.rename(columns={'id':'pricing_id', 'app_id':'id' ,'title':'pricing_plan_title'}, inplace=True)





print(pricing_plan_features_df.shape)
pricing_plan_features_df.head()

(23990, 3)


,pricing_id,id,pricing_plan_feature
0,88ed7ece-f8f7-48f0-af85-e8d0ab4a1d2d,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,5 Entries
1,88ed7ece-f8f7-48f0-af85-e8d0ab4a1d2d,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,Custom HTML
2,7238abeb-b83b-44a9-bc0f-922b6ccdf90f,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,40% OFF YEARLY PLANS THRU CYBER MONDAY!
3,002eac81-2169-452d-8491-7125b6d3855f,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,For stores on Basic Shopify plan
4,002eac81-2169-452d-8491-7125b6d3855f,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,All current and future apps Included


In [38]:
#grouping pricing plan features on pricing id & consolidating duplicate pricing_plan features
#app_id lost as a result of this transformation
grouped_pricing_plan_features = pricing_plan_features_df.groupby("pricing_id", as_index=False).agg(
    {'pricing_plan_feature': lambda ppf : ' || '.join(ppf)})
    
#testing result of .agg
print(grouped_pricing_plan_features.shape)
grouped_pricing_plan_features.head()

(6087, 2)


,pricing_id,pricing_plan_feature
0,000843f0-bc81-4dea-a120-20ffa406090c,Fulfill multiple orders & create shipping labe...
1,000ad3c8-2091-4425-bb22-9100940871cc,One click enable/disable || Choose 5+ icons ||...
2,0010d6e7-7431-4c63-a224-ac73a0f2e887,"Everything in Business, plus: || Phone, Email ..."
3,0013f8ba-8e2a-4f35-a00e-40edba26b35f,"1,500 Average Impressions/Month || Unlimited a..."
4,002eac81-2169-452d-8491-7125b6d3855f,For stores on Basic Shopify plan || All curren...


In [56]:
#merging pricing & pricing_plans to create a hybrid table
pricing = pricing_plans_df.merge(grouped_pricing_plan_features, how='left', on=['pricing_id'])

print(pricing.shape)
pricing.head()

pricing.dtypes

(8514, 5)


pricing_id              object
id                      object
pricing_plan_title      object
price                   object
pricing_plan_feature    object
dtype: object

In [ ]:
#reviews_df - not used at this time

In [62]:
#Creation of app summary dataframe
#consists of: apps_df + grouped_categories + grouped_key_benefits
one_df = apps_df
two_df = one_df.merge(grouped_categories, how = 'left', on = 'id', suffixes = ('','_2'))
app_summary_df = two_df.merge(grouped_key_benefits, how = 'left', on = 'id', suffixes =('','_3'))


In [58]:
#Creation of Pricing Summary Data Frame
#I wanted to consolidate this table to have one id with multiple pricing id, price, pricing_plans but
#could not figure out why this error was throwing
pricing_summary_df = pricing.groupby("id", as_index=False).agg(
    {'pricing_plan_title': lambda ppt : ' || '.join(ppt),
     'price': lambda p : ' || '.join(p),
    'pricing_plan_feature': lambda ppf : ' || '.join(ppf)})

/Users/msmith/opt/anaconda3/lib/python3.9/site-packages/pandas/core/apply.py:436: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  key: obj._gotitem(key, ndim=1).agg(how) for key, how in arg.items()


TypeError: sequence item 0: expected str instance, float found

In [67]:
#reviewing completed app_summary dataframe
print(app_summary_df.shape)
app_summary_df.head()

(4750, 15)


,id,url,app_title,developer,developer_link,icon,rating,reviews_count,description_raw,description,tagline,pricing_hint,app_category,benefit_title,benefit_description
0,9e4748a9-7eda-4814-83b6-0537d44152b1,https://apps.shopify.com/translate,Panda Language Translate,Panda Apps,https://apps.shopify.com/partners/ittechzilla,https://apps.shopifycdn.com/listing_images/269...,4.7,379,"<div class=""block app-listing__detailed-descri...",Panda Language Translation improve your busine...,Translate your store into multiple languages,7-day free trial,Store design,Translate your store easily || Powerful transl...,Add 100+ languages in your store. Increase you...
1,d1476138-a608-4bb9-8d39-b30f3ca7617d,https://apps.shopify.com/instant-brand-page,Instant Brand Page,Low Fruit Solutions PTY LTD,https://apps.shopify.com/partners/lowfruitsolu...,https://apps.shopifycdn.com/listing_images/51d...,4.9,13,"<div class=""block app-listing__detailed-descri...",Having a brand page gives your shoppers a quic...,A-Z Brand Index Page and Favourites Slider,10-day free trial,Store design,Quick and Easy Install || Use Custom brand ima...,No coding skills or changes required! Customis...
2,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,https://apps.shopify.com/powr-faq,FAQ Accordion | Help Center,POWR.io,https://apps.shopify.com/partners/powr-io,https://apps.shopifycdn.com/listing_images/86c...,4.5,202,"<div class=""block app-listing__detailed-descri...","Create, organize and display Frequently Asked ...","FAQ page, FAQ accordion menu for product Info ...",NaN,"Store design, Customer support",FAQ Accordion Product Info || FAQ Accordion Re...,Add a custom FAQ accordion with answers to com...
3,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,https://apps.shopify.com/mps-promote-me,Promote Me | Many apps in one,MakeProSimp,https://apps.shopify.com/partners/makeprosimp,https://apps.shopifycdn.com/listing_images/5dc...,4.9,18,"<div class=""block app-listing__detailed-descri...",Promote Me app is a bundle of apps that includ...,"Spin Wheel,Currency Converter,Quick ATC Button...",10-day free trial,"Store design, Sales and conversion optimization",Improve Conversions & Revenue || Create Emerge...,"Wheel of Fortune, Currency Converter, Animated..."
4,7aac2a1f-ff03-4f38-aeb7-7619403a6f05,https://apps.shopify.com/installify-boost-mobi...,Instalify,Plobal Apps,https://apps.shopify.com/partners/plobal-apps,https://apps.shopifycdn.com/listing_images/737...,0.0,0,"<div class=""block app-listing__detailed-descri...",We are trusted by hundreds of Shopify and Shop...,Supercharge Your Mobile App Installs,7-day free trial,Store design,Drive Organic App Installs || Quick & Easy Set...,Organically keep increasing your mobile app in...


In [65]:
#checking for missing values on app_summary dataframe
missing = pd.concat([app_summary_df.isnull().sum(), 100 * app_summary_df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing = missing.sort_values(by=['count'], ascending = False)

print(missing)

                     count          %
pricing_hint          1980  41.684211
benefit_title          441   9.284211
benefit_description    441   9.284211
id                       0   0.000000
url                      0   0.000000
app_title                0   0.000000
developer                0   0.000000
developer_link           0   0.000000
icon                     0   0.000000
rating                   0   0.000000
reviews_count            0   0.000000
description_raw          0   0.000000
description              0   0.000000
tagline                  0   0.000000
app_category             0   0.000000


In [69]:
#reviewing pricing shape & status -- I would like to replace this with the pricing_summary_df above
#doing so would result in ~4000 records and no duplicate apps
print(pricing.shape)
pricing.head()

(8514, 5)


,pricing_id,id,pricing_plan_title,price,pricing_plan_feature
0,88ed7ece-f8f7-48f0-af85-e8d0ab4a1d2d,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,Free,Free,5 Entries || Custom HTML
1,f88514b2-ad9c-4625-adc8-2643625168f1,9e4748a9-7eda-4814-83b6-0537d44152b1,NaN,$7.99/month,NaN
2,002eac81-2169-452d-8491-7125b6d3855f,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,BASIC,$2.95/month,For stores on Basic Shopify plan || All curren...
3,7238abeb-b83b-44a9-bc0f-922b6ccdf90f,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,Starter,$4.99/month,40% OFF YEARLY PLANS THRU CYBER MONDAY! || No ...
4,2a8a95dd-5d6e-4bda-8605-d29816b0ecd5,d1476138-a608-4bb9-8d39-b30f3ca7617d,NaN,$5.99/month,NaN


In [70]:
#checking for missing pricing records
missing_prices = pd.concat([pricing.isnull().sum(), 100 * pricing.isnull().mean()], axis=1)
missing_prices.columns=['count', '%']
missing_prices = missing_prices.sort_values(by=['count'], ascending = False)

print(missing_prices)

                      count          %
pricing_plan_feature   2427  28.505990
pricing_plan_title     2341  27.495889
pricing_id                0   0.000000
id                        0   0.000000
price                     0   0.000000
